In [1]:
import argparse
from ase import Atoms
from ase.io import read, write
from ase.optimize import *
from ase import units
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution, ZeroRotation, Stationary
from ase.md.langevin import Langevin
from ase.io.trajectory import Trajectory
from os.path import splitext
from ase.visualize import view
from ase.md.verlet import VelocityVerlet
import numpy as np
from NNCalculator.NNCalculator import *
import time

import ase
from ase import io
from ase.visualize import view

import matplotlib.pyplot as plt

from tqdm import tqdm

def dihedral3(p):
    b = p[:-1] - p[1:]
    b[0] *= -1
    v = np.array( [np.cross(v,b[1]) for v in [b[0], b[2]] ] )
    # Normalize vectors
    v /= np.sqrt(np.einsum('...i,...i', v, v)).reshape(-1,1)
    return np.degrees(np.arccos( v[0].dot(v[1]) ))

def cond(x):
    return 6 if x > 10 else  x

def vis_pdb(pdb):
    atomic_numbers = pdb.get_atomic_numbers()
    print(atomic_numbers)
    #  hack to get around the color scheme and atom type names
    #. eg. CYA for alpha carbon types
    pdb.set_atomic_numbers([ cond(_)
                            for _ in atomic_numbers])
    return view(pdb, viewer="x3d")

/home/himmelreich/.conda/envs/pyCHARMM/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/himmelreich/.conda/envs/pyCHARMM/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/himmelreich/.conda/envs/pyCHARMM/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/himmelreich/.conda/envs/pyCH

In [2]:
initial_pdb = io.read('/home/himmelreich/PCProject/pyCHARMM-Workshop/1FirstExample/analysispdb/b48733a2-a7c1-4f1a-a342-663bc739540f-frame1935.pdb')
initial_pdb

Atoms(symbols='CaH3CONHCaHCH2OgHgCONHCaH3', pbc=False, atomtypes=..., bfactor=..., occupancy=..., residuenames=..., residuenumbers=...)

#

In [3]:
# INPUT = "min.xyz"
OUTPUT = "opt_min.xyz"

In [4]:
# atoms = read(INPUT)


In [5]:
# vis_pdb(atoms)

In [6]:
vis_pdb(initial_pdb)

[ 20   1   1   1   6   8   7   1  20   1   6   1   1 118  80   6   8   7
   1  20   1   1   1]


In [7]:
atoms = initial_pdb

In [8]:
calc = NNCalculator(
    checkpoint="models/Ac-Ala3-NHMe-coff10-elec-disp-wf53-cw14-alldata-b", #load the model you want to used
    atoms=atoms,
    charge=0,
    F=128,
    K=64,
    num_blocks=5,
    num_residual_atomic=2,
    num_residual_interaction=3,
    num_residual_output=1,
    sr_cut=10.0,
    use_electrostatic=True,
    use_dispersion=True,
    s6=1.0000,                    #s6 coefficient for d3 dispersion, by default is learned
    s8=2.3550,                    #s8 coefficient for d3 dispersion, by default is learned
    a1=0.5238,                    #a1 coefficient for d3 dispersion, by default is learned
    a2=3.5016)                   #a2 coefficient for d3 dispersion, by default is learned)






/home/himmelreich/.conda/envs/pyCHARMM/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Restoring parameters from models/Ac-Ala3-NHMe-coff10-elec-disp-wf53-cw14-alldata-b


In [9]:
#attach the calculator object to the atoms object
atoms.set_calculator(calc)

#optimize
opt = BFGS(atoms)
opt.run(10**6)

#write output file
write(OUTPUT, atoms)

      Step     Time          Energy         fmax
BFGS:    0 17:02:08        1.530435        7.3682


In [11]:
TEMP = 298
CHARGE = 0
STEPS = 10**5
TIMESTEP = 0.01
FRICTION = 0.02
INTERVAL = 20
filename = "dynamics"

In [12]:
#run an optimization
BFGS(atoms).run(fmax=0.01)

      Step     Time          Energy         fmax
BFGS:    0 17:02:22        1.530435        7.3682
BFGS:    1 17:02:22        0.799056        2.2725
BFGS:    2 17:02:22        0.476366        3.0016
BFGS:    3 17:02:22        0.287272        2.0868
BFGS:    4 17:02:22        0.113883        1.0546
BFGS:    5 17:02:22        0.013538        1.1328
BFGS:    6 17:02:22       -0.054400        0.9655
BFGS:    7 17:02:22       -0.119517        0.9597
BFGS:    8 17:02:22       -0.170646        1.0796
BFGS:    9 17:02:22       -0.227990        1.1084
BFGS:   10 17:02:22       -0.291316        1.0253
BFGS:   11 17:02:22       -0.343181        0.6472
BFGS:   12 17:02:22       -0.403316        0.9818
BFGS:   13 17:02:22       -0.451778        1.2347
BFGS:   14 17:02:22       -0.508390        1.1378
BFGS:   15 17:02:22       -0.556239        1.0011
BFGS:   16 17:02:23       -0.586171        1.2295
BFGS:   17 17:02:23       -0.620071        1.3148
BFGS:   18 17:02:23       -0.667040        1.2043
B

True

In [ ]:
# Set the momenta corresponding to a temperature T
MaxwellBoltzmannDistribution(atoms, TEMP * units.kB)
ZeroRotation(atoms)
Stationary(atoms)

# define the algorithm for MD: here Langevin alg. with with a time step of 0.1 fs,
# the temperature T and the friction coefficient to 0.02 atomic units.
#dyn = Langevin(atoms, TIMESTEP * units.fs, TEMP * units.kB, FRICTION)
dyn = VelocityVerlet(atoms, args.timestep * units.fs)


def printenergy(a=atoms):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    print('Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.3feV' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin))
    
# save the positions of all atoms after every Xth time step.
traj = Trajectory(str(TEMP)+ 'K_md_' + filename + '.traj', 'w', atoms)

"""
#equilibration
for i in range(10000):
    if i%100 == 0:
        print("Equilibration Step: ", i)
    dyn.run(1)

"""
start_time = time.time()
# run the dynamics
for i in tqdm(range(STEPS)):
    dyn.run(1)
    if i%INTERVAL == 0:
        #epot = atoms.get_potential_energy() / len(atoms)
        #ekin = atoms.get_kinetic_energy() / len(atoms)
        # print("Production Step: ", i)
        traj.write()

end_time = time.time()


# Calculate elapsed time
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)
print("Time per step: ", elapsed_time / STEPS)

In [ ]:
# traj.__dict__

In [ ]:
intraj = Trajectory("298K_md_dynamics.traj")

#  Analysis

In [ ]:
coords = []
for i, atoms in enumerate(intraj):
    # print(i, atoms.positions)
    coords.append(atoms.positions)

In [ ]:
psi_idxs = [6,8,10,16]
# print(protein.getNames()[psi_idxs])
psidihs = [dihedral3(coords[i][psi_idxs]) for i
        in range(len(coords))]

plt.plot(range(len(psidihs)), psidihs)
plt.xlabel("Time (ps)")
plt.ylabel("$\psi$ Angle ($^{\circ}$)")


In [ ]:
phi_idxs = [4,6,8,10]
phidihs = [dihedral3(coords[i][phi_idxs]) for i
        in range(len(coords))]

plt.plot(range(len(phidihs)), phidihs)
plt.xlabel("Time (ps)")
plt.ylabel("$\psi$ Angle ($^{\circ}$)")